In [7]:
import sys, os, re, time
import numpy as np
import pandas as pd
import defusedxml.ElementTree as ET
from collections import defaultdict

if '../python' not in sys.path: sys.path.append('../python')
if '../cpp/pyfit-build' not in sys.path: sys.path.append('../cpp/pyfit-build')

from loadconfig import CONFIG    
from fittransformer_so import FitTransformer

ModuleNotFoundError: No module named 'loadconfig'

In [1]:
conda_prefix = '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev'
install_prefix = '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages'

['cmake', '-Spyfitparquet/cpp', '-Bpyfit-build',
 f'-DCMAKE_PREFIX_PATH={conda_prefix}',
 f'-DCMAKE_INSTALL_PREFIX={install_prefix}']

['cmake',
 '-Spyfitparquet/cpp',
 '-Bpyfit-build',
 '-DCMAKE_PREFIX_PATH=/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev',
 '-DCMAKE_INSTALL_PREFIX=/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages']

In [7]:
import os

def find_pyinstall_dir(conda_prefix, idir='site-packages'):
    for root, dirs, files in os.walk(conda_prefix):
        if idir in dirs: 
#             print(f'{root} - {idir}')
            print(os.path.relpath(os.path.join(root, idir), conda_prefix))
            return os.path.join(root, idir)
        
    return None

find_pyinstall_dir(conda_prefix)

lib/python3.8/site-packages


'/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages'

In [13]:
# from subprocess import call
# call([sys.executable, 'scriptname.py'],
#      cwd=os.path.join(dir, 'packagename'))
# import sys
# os.path.abspath(os.path.join(sys.executable, '..', '..'))

import sysconfig
print(sysconfig.get_paths()["purelib"])
print(sysconfig.get_paths())

/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages
{'stdlib': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8', 'platstdlib': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8', 'purelib': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages', 'platlib': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/lib/python3.8/site-packages', 'include': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/include/python3.8', 'platinclude': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/include/python3.8', 'scripts': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev/bin', 'data': '/Users/ajdonich/opt/miniconda3/envs/pyfitparquet-dev'}


In [ ]:
timestamps = [pd.to_datetime(t).tz_localize(None) for t in 
              ['2021-01-13T20:05:05Z', '2021-01-13T20:05:06Z', 
               '2021-01-13T20:05:09Z']]

df = pd.DataFrame({'one': [-1, np.nan, 2.5],
                   'two': ['foo', 'bar', None],
                   'three': [True, False, True],
                   'four': timestamps},
                   index=list('abc'))

In [ ]:
display(df)
# df.to_parquet(path='example.parquet', engine='pyarrow')
print(df.dtypes)


In [47]:
tags = ['something', 'AverageHeartRateBpm']

icaps = [i for i, ltr in enumerate(tags[-1]) if ltr.isupper()] + [len(tags[-1])]
parts = [tags[-1][icaps[j-1]:icaps[j]].lower() for j in range(1, len(icaps))]
if parts[-1] in ['bpm', 'meters']: units = parts[-1]; parts.pop()
'_'.join(parts)

'average_heart_rate'

In [34]:
def striptag(clarkname):
    # Tcx xnode.tag generally includes xmlns, thus appears as: '{xmlns}tagname'
    matchobj = re.match(r'^(\{(?P<xmlns>\w+:.+)\})?(?P<tagname>[:\w]+)', clarkname)
    assert matchobj, f"Unable to appropriately strip element tag: '{clarkname}'"
    return matchobj.group('xmlns'), matchobj.group('tagname')

In [35]:
striptag('xsd:dateTime')

(None, 'xsd:dateTime')

In [29]:
a = """45_min_HIIT_Hills_Ride_with_Sam_Yo.tcx:<TrainingCenterDatabase 
xmlns:ns2="http://www.garmin.com/xmlschemas/UserProfile/v2" 
xmlns:ns3="http://www.garmin.com/xmlschemas/ActivityExtension/v2" 
xmlns:ns5="http://www.garmin.com/xmlschemas/ActivityGoals/v1" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xmlns="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
xsi:schemaLocation="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd">

45_min_Power_Zone_Ride_with_Matt_Wilpers.tcx:<TrainingCenterDatabase 
xmlns="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xmlns:ns2="http://www.garmin.com/xmlschemas/UserProfile/v2" 
xmlns:ns3="http://www.garmin.com/xmlschemas/ActivityExtension/v2" 
xmlns:ns5="http://www.garmin.com/xmlschemas/ActivityGoals/v1" 
xsi:schemaLocation="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd">

60_min_Power_Zone_Endurance_Ride_with_Christine_D_Ercole.tcx:<TrainingCenterDatabase 
xmlns="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" 
xmlns:ns2="http://www.garmin.com/xmlschemas/UserProfile/v2" 
xmlns:ns3="http://www.garmin.com/xmlschemas/ActivityExtension/v2" 
xmlns:ns5="http://www.garmin.com/xmlschemas/ActivityGoals/v1" 
xsi:schemaLocation="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd">

TeamScream.tcx:<TrainingCenterDatabase 
xsi:noNamespaceSchemaLocation="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
http://www.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd" 
xmlns:ns5="http://www.garmin.com/xmlschemas/ActivityGoals/v1" 
xmlns:ns3="http://www.garmin.com/xmlschemas/ActivityExtension/v2" 
xmlns:ns2="http://www.garmin.com/xmlschemas/UserProfile/v2" 
xmlns="http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2" 
xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">"""

xnamespaces = set()
for line in a.split('\n'):
    if re.match(r'.+=.+', line):
        matchobj = re.match(r'.*\"(.+)[\n\"]', line)
        xnamespaces.add(matchobj.group(1))

print(xnamespaces)
for ns in xnamespaces:
    matchobj = re.match(r'http://www.garmin.com/xmlschemas/(\w+)/(\w+)', ns)
    if matchobj: 
        uri = f'https://www8.garmin.com/xmlschemas/{matchobj.group(1)}{matchobj.group(2)}.xsd'
        print(uri)
#         !wget $uri
    
    #http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2
    #https://www8.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd
    


{'http://www.garmin.com/xmlschemas/UserProfile/v2', 'http://www.garmin.com/xmlschemas/ActivityExtension/v2', 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2', 'http://www.garmin.com/xmlschemas/ActivityGoals/v1', 'http://www.w3.org/2001/XMLSchema-instance'}
https://www8.garmin.com/xmlschemas/UserProfilev2.xsd
https://www8.garmin.com/xmlschemas/ActivityExtensionv2.xsd
https://www8.garmin.com/xmlschemas/TrainingCenterDatabasev2.xsd
https://www8.garmin.com/xmlschemas/ActivityGoalsv1.xsd


In [6]:
xnamespaces = set()
data_dir = '../../data'

for file in os.listdir(data_dir):
#{
    matchtcx = re.match('(\w+).(tcx|TCX)$', file)            
    if matchtcx: 
        initial = time.time()
        tcxfile = f"{data_dir}/{file}"
        
        with open(tcxfile) as f_handle:
            xmlstring = f_handle.read().lstrip()
            for xnode in ET.fromstring(xmlstring).iter():
                xmlns, tagname = striptag(xnode.tag)
                xnamespaces.add(xmlns)
#}

xnamespaces

{'http://www.garmin.com/xmlschemas/ActivityExtension/v2',
 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'}

In [5]:
def recurse_tree(xiter,  pad=''):
#{
    xnode = next(xiter)
    tagname = xnode.tag
    print(pad + tagname + " " + str(xnode.attrib))

    for _ in range(len(xnode)):
        recurse_tree(xiter, f'  {pad}')
#}

# Deal w/Clark notation
def striptag(clarkname):
    # Tcx xnode.tag generally includes xmlns, thus appearing as: '{xmlns}tagname'
    matchobj = re.match(r'^(\{(?P<xmlns>\w+:.+)\})?(?P<tagname>\w+)', clarkname)
    assert matchobj, f"Unable to appropriately strip element tag: '{clarkname}'"
    return matchobj.group('xmlns'), matchobj.group('tagname')

def analyze_tree(xiter, compoundtags=None, tagstack=None):
#{
    #if tagmap is None: tagmap = set()
    #if attribmap is None: attribmap = defaultdict(set)
    
    if tagstack is None: tagstack = []
    if compoundtags is None: compoundtags = set()

    xnode = next(xiter)
    xmlns, tagname = striptag(xnode.tag)
    
    tagstack.append(tagname)
    comptag = '-'.join(tagstack)
    for k in xnode.attrib.keys():
        xmlns, kname = striptag(k)
        compoundtags.add(f'{comptag}-{kname}')
        
    if xnode.text and xnode.text.rstrip(): compoundtags.add(comptag)
        
    for _ in range(len(xnode)):
        analyze_tree(xiter, compoundtags, tagstack)
    
    tagstack.pop()
#}

In [ ]:
# xiter = ET.parse('example.xml').iter()
xiter = ET.parse('../../data/TeamScream.tcx').iter()

compoundtags = set()
analyze_tree(xiter, compoundtags)
compoundtags

In [ ]:
CONFIG

In [ ]:
parquet_maker = PyFitParquet()
#parquet_maker.data_to_parquet('../../data/')
df = parquet_maker.tcx_to_parquet('../../data/TeamScream.tcx', './TeamScream.parquet')
display(df)
display(df[:60])

In [ ]:
pfile = './TeamScream.parquet'

pdframe = pd.read_parquet(pfile, engine='pyarrow')
print(pfile + ":")
display(pdframe)

In [ ]:
#     def append_position(self, xiter, nlen):
#         for _ in range(nlen):
#             fnode = next(xiter)
#             schema, tagname = self.striptag(fnode)
#             if tagname == "LatitudeDegrees": 
#             elif tagname == "LongitudeDegrees": 

#         <xsd:element name="LatitudeDegrees" type="DegreesLatitude_t"/>
#         <xsd:element name="LongitudeDegrees" type="DegreesLongitude_t"/>
        
#     def insert_altitude
#     def insert_distance
#     def insert_heart_rate
#     def insert_cadence
#     def insert_sensor_state
#     def insert_extensions

def append_trackpoint(self, xiter, nlen):
    #{
        nfields = 0
        for ti in range(nlen):
            fnode = next(xiter)
            xmlns, tagname = striptag(fnode)
            if tagname == "Time":
                self.timestamp = pd.to_datetime(fnode.text).tz_localize(None)
                if self.excludeflags["exclude_timestamp_values"]: continue
                append_mesg_fields('Trackpoint'):
                append_field_fields()
                nfields += 1
                
                
#             elif tagname == "Position":
#                 for pi in range(len(fnode)):
#                     fnode = next(xiter)
#                     xmlns, tagname = self.striptag(fnode)
#                     if tagname == "LatitudeDegrees": 
#                     elif tagname == "LongitudeDegrees": 
                
#             elif tagname == "AltitudeMeters": 
#             elif tagname == "DistanceMeters":
#             elif tagname == "HeartRateBpm":
#             elif tagname == "Cadence":
#             elif tagname == "SensorState":
#             elif tagname == "Extensions":
                
            print("  " + tagname + " : " + fnode.text.rstrip())
    #}
    


# try: 
# #{
#     xiter = ET.parse(tcx_fname).iter()

#     # TODO: find/get Creator tag first to init "file_id" fields

#     debugcounter = 0
#     while True:
#         xnode = next(xiter)
#         schema, tagname = striptag(xnode)

#         print(tagname)
#         if tagname == 'Trackpoint':
#             append_trackpoint(xiter, len(xnode))
#         elif tagname == 'Lap':
#             append_lap(xiter, len(xnode))

#         # Tmp loop breaker
#         if debugcounter > 200: break
#         debugcounter += 1
# #}
# except StopIteration: pass

In [ ]:
tree = ET.parse('example.xml')
for elem in tree.iter(): print(elem)
print()

In [ ]:
counter = 0


tree = ET.parse('../../data/TeamScream.tcx')
# tree = ET.parse('example.xml')
xiter = tree.iter()

try: 
#{
    while True:
        xnode = next(xiter)
        schema, tagname = parsetag(xnode)
        print(f'{tagname} : {len(xnode)}')
        
#         if not schema: print(tagname)
#         else:  print(schema + " : " + tagname)

        print(tagname)
        if tagname == 'Trackpoint':
            insert_trackpoint(xiter, len(xnode))
            
        # Tmp loop breaker
        if counter > 200: break
        counter += 1
#}
except StopIteration: pass
    
#     tag = re.sub(r'^\{.+\}', '', elem.tag)
#     text = elem.text.rstrip() if elem.text else ''
#     print(f"Tag: {tag}, Value: {text}, Attrib: {elem.attrib}")


In [ ]:
tree = ET.parse('../../data/TeamScream.tcx')
# tree = ET.parse('../../data/45_min_HIIT_Hills_Ride_with_Sam_Yo.tcx')
# enode = tree.iter()
# print(enode)
# for elem in tree.iter():
#     print elem

counter = 0
for elem in tree.iter():
    if counter > 125: break
    tag = re.sub(r'^\{.+\}', '', elem.tag)
    text = elem.text.rstrip() if elem.text else ''
    print(f"Tag: {tag}, Value: {text}, Attrib: {elem.attrib}")
    counter += 1




In [ ]:
# for child in root:
#     print(child.tag, child.attrib)